# Compute Functions and Jobs
__________________
The Compute module provides scalable compute capabilities to parallelize your computations. Compute enables users to package and execute your Python code within nodes hosted on Descartes Lab's cloud infrastructure. These nodes offer the ability to access imagery at extremely high rates of throughput to execute computations over nearly any spatio-temporal scale. 

Within the compute module, there are two primary objects that we work with:

 * [**Function:**](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) dynamically created, serverless functions containing user submitted, compiled code that you can submit many jobs to.
 * [**Job:**](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Job) submitted request for a single invocation of a created Function. 

Let's start by importing the compute module and creating a test Function object. 

In [ ]:
from descarteslabs.compute import Function, Job

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import Blob, properties as p

Next, we'll create a very basic `hello_world` function that imports a common geospatial Python package and returns a given argument.

In [ ]:
def hello(i):
    print(f"Hello, {i}")
    return f"hello {i}"

To create the Function object, we simply need to call `Function()` and specify the desired parameters. The recommended minimum parameters are your function, a name, and the image that will be used to build the Function environment.  Some common attributes used to better customize the performance of your scalable compute object include: 
 * cpus = Number of CPUs requested for a single job
 * memory = max memory available for each job
 * maximum_concurrency = max number of jobs to run in parallel
 * timeout = max length a job can run in seconds
 * retry_count = max number of times a job can be retried
 * requirements = list of Python dependencies required by this function

For other options, please see the [Compute documentation](https://docs.descarteslabs.com/guides/compute.html).

In [ ]:
print("creating function")
async_func = Function(
    hello,
    name="my-compute-hello",
    image="python3.9:latest",
    cpus=0.25,
    memory=512,
    maximum_concurrency=1,
    timeout=600,
    retry_count=0,
)
async_func.save()

Once the Function is built, we will test it by creating and submitting a Job. There are several ways you can submit jobs to a function:
 * `async_func(args)` - Pass arguments directly to the compute.Function
 * [`async_func.map()`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function.map) - Submit multiple jobs efficiently (discussed in more detail within the next example - "02_Create_Imagery.ipynb")

In [ ]:
# invoke the function
print("submitting a job")
job = async_func("Hello from my function!")
# print the job result and logs
print("waiting for the job to complete")
job.wait_for_completion()

### Waiting for Completion
Now that we've mapped our arguments to `Job`s, we can wait for our `Function` to complete by either navigating to [app.descarteslabs.com/monitor](https://app.descarteslabs.com/monitor) or programmatically via:

In [ ]:
from IPython.display import IFrame

IFrame("https://app.descarteslabs.com/monitor", width=700, height=350)

### Integration with DL Storage - Logging and Results
Once your `Function` is completed, you can access the build logs as well as the results of each `Job` through the `Storage` module. These three items are outlined below:

* Function Build Logs:

    `logs/org:user-hash/function-id`

* Job Execution Logs:

    `logs/org:user-hash/function-id/job-id`

* Job Results:
        
    `compute/org:user-hash/function-id/job-id`
    

First, we must find the current active `Organization` and `User Hash`. These, combined with our `Function` ID, allows us to search `Blob`s for our results and logs:

In [ ]:
orgname = dl.auth.Auth().payload["org"]
user_hash = dl.auth.Auth().namespace

print(f"Function Logs for {async_func.id}")
function_log = Blob.get(f"logs/{orgname}:{user_hash}/{async_func.id}")
print(function_log)

print(f"Results for {async_func.id}")
for b in (
    Blob.search()
    .filter(p.namespace == f"{orgname}:{user_hash}")
    .filter(p.name.startswith(async_func.id))
    .filter(p.storage_type == "compute")
):
    print(f"ID: {b.id}")
    print(b.data())
    print("\n")

print(f"Job Execution Logs for {async_func.id}")
for b in (
    Blob.search()
    .filter(p.namespace == f"{orgname}:{user_hash}")
    .filter(p.name.startswith(f"{async_func.id}/"))
    .filter(p.storage_type == "logs")
):
    print(b.id)
    print(b.data())
    print("\n")

### Listing and Filtering Functions
You can also list and filter your previously created `Function` objects. Here we will find all of our `Function`s created today:

In [ ]:
from datetime import datetime

today = datetime.today().strftime("%Y-%m-%d")

for func in Function.list().filter(p.creation_date > today):
    print(func.id)
    print(func.creation_date)
    print(func.status)
    print(len(list(func.jobs)))

In [ ]:
# Get active functions by specifying the "running" status
active_funcs = [{"name": f.name, "id": f.id} for f in Function.list(status="running")]
active_funcs

In [ ]:
# Filter Functions by name prefixes
funcs = Function.list()
ndvi_funcs = [f.name for f in funcs if f.name.startswith("my")]
ndvi_funcs

### Deleting Functions
In this cell we will find and delete all `Function`s created before today's date:

In [ ]:
Function.get("ee71d78b-5455-449d-abfb-28d2b4e2ac1a")

In [ ]:
for func in Function.list().filter(p.creation_date < today):
    print(f"Deleting {func.id}\nCreated at: {func.creation_date}")
    func.delete()